In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk import word_tokenize
import string

In [2]:
import w1_unittest

In [3]:
dataframe_emails = pd.read_csv('emails.csv')
dataframe_emails.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [4]:
#exploring the dataset
print(f"Number of emails: {len(dataframe_emails)}")
print(f"Proportion of spam emails: {dataframe_emails.spam.sum()/len(dataframe_emails):.4f}")
print(f"Proportion of ham emails: {1-dataframe_emails.spam.sum()/len(dataframe_emails):.4f}")

Number of emails: 5728
Proportion of spam emails: 0.2388
Proportion of ham emails: 0.7612


In [5]:
def preprocess_emails(df):
    # Shuffles the dataset
    df = df.sample(frac = 1, ignore_index = True, random_state = 42)
    # Removes the "Subject:" string, which comprises the first 9 characters of each email. Also, convert it to a numpy array.
    X = df.text.apply(lambda x: x[9:]).to_numpy()
    # Convert the labels to numpy array
    Y = df.spam.to_numpy()
    return X, Y

In [6]:
X, Y = preprocess_emails(dataframe_emails)

In [10]:
def preprocess_text(X):
    """
    Preprocesses a collection of text data by removing stopwords and punctuation.
    """
    # Make a set with the stopwords and punctuation
    stop = set(stopwords.words('english') + list(string.punctuation))

    # The next lines will handle the case where a single email is passed instead of an array of emails.
    if isinstance(X, str):
        X = np.array([X])

    # The result will be stored in a list
    X_preprocessed = []

    for i, email in enumerate(X):
        email = np.array([i.lower() for i in word_tokenize(email) if i.lower() not in stop]).astype(X.dtype)
        X_preprocessed.append(email)
        
    if len(X) == 1:
        return X_preprocessed[0]
    return X_preprocessed


In [11]:

X_treated = preprocess_text(X)

In [13]:
#Splitting data into train/test
TRAIN_SIZE = int(0.80*len(X_treated)) # 80% of the samples will be used to train.
X_train = X_treated[:TRAIN_SIZE]
Y_train = Y[:TRAIN_SIZE]
X_test = X_treated[TRAIN_SIZE:]
Y_test = Y[TRAIN_SIZE:]

In [15]:
def get_word_frequency(X,Y):
    """
    Calculate the frequency of each word in a set of emails categorized as spam (1) or not spam (0).
    
    """
    word_dict = {}

    num_emails = len(X)

    for i in range(num_emails):
        email = X[i] 
        cls = Y[i] 
        email = set(email) 
        for word in email:
            if word not in word_dict.keys():
                word_dict[word] = {'spam': 1, 'ham': 1}
            if cls == 0:    
                word_dict[word]['ham'] += 1
            if cls == 1:
                word_dict[word]['spam'] += 1
    
    return word_dict

In [18]:
# Building the word_frequency dictionary using the training set. 
word_frequency = get_word_frequency(X_train,Y_train)

In [19]:
# Counting the spam and ham emails
class_frequency = {'ham': sum(Y_train == 0), 'spam': sum(Y_train == 1)}

In [22]:
def prob_word_given_class(word, cls, word_frequency, class_frequency):
    """
    Calculate the conditional probability of a given word occurring in a specific class.

    """
    # Get the amount of times the word appears with the given class (class is stores in spam variable)
    amount_word_and_class = word_frequency[word][cls]
    p_word_given_class = amount_word_and_class/class_frequency[cls]
    
    return p_word_given_class

In [27]:
def prob_email_given_class(treated_email, cls, word_frequency, class_frequency):
    """
    Calculate the probability of an email being of a certain class (e.g., spam or ham) based on treated email content.

    """

    prob = 1

    for word in treated_email:
        
        if word in word_frequency.keys(): 

            prob *= prob_word_given_class(word, cls, word_frequency = word_frequency, class_frequency = class_frequency)

    return prob

In [30]:
def naive_bayes(treated_email, word_frequency, class_frequency, return_likelihood = False):    
    """
    Naive Bayes classifier for spam detection.

    This function determines whether an email is likely to be spam (1) or not spam (0) using the Naive Bayes algorithm.
    It relies on the conditional probabilities associated with the treated email being classified as spam or not spam,
    along with the prior probabilities of spam and not spam classes. The ultimate classification is determined by comparing these calculated probabilities.

    """

    prob_email_given_spam = prob_email_given_class(treated_email, 'spam', word_frequency, class_frequency)

    prob_email_given_ham = prob_email_given_class(treated_email, 'ham', word_frequency, class_frequency)

    p_spam = class_frequency['spam']/(class_frequency['spam']+class_frequency['ham'])
    
    p_ham = class_frequency['ham']/(class_frequency['spam']+class_frequency['ham'])

    spam_likelihood = p_spam * prob_email_given_spam

    ham_likelihood = p_ham * prob_email_given_ham
    
    if return_likelihood == True:
        return (spam_likelihood, ham_likelihood)
    
    elif spam_likelihood >= ham_likelihood:
        return 1
    else:
        return 0

In [33]:
def get_true_positives(Y_true, Y_pred):
    """
    Calculate the number of true positive instances in binary classification.

    """
    if len(Y_true) != len(Y_pred):
        return "Number of true labels and predict labels must match!"
    n = len(Y_true)
    true_positives = 0
    for i in range(n):
        true_label_i = Y_true[i]
        predicted_label_i = Y_pred[i]
        if true_label_i == 1 and predicted_label_i == 1:
            true_positives += 1
    return true_positives
        
def get_true_negatives(Y_true, Y_pred):
    """
    Calculate the number of true negative instances in binary classification.

    """
    if len(Y_true) != len(Y_pred):
        return "Number of true labels and predict labels must match!"
    n = len(Y_true)
    true_negatives = 0
    for i in range(n):
        true_label_i = Y_true[i]
        predicted_label_i = Y_pred[i]
        if true_label_i == 0 and predicted_label_i == 0:
            true_negatives += 1
    return true_negatives
        

In [34]:

Y_pred = []
for email in X_test:
    prediction = naive_bayes(email, word_frequency, class_frequency)
    Y_pred.append(prediction)
print(f"Y_test and Y_pred matches in length? Answer: {len(Y_pred) == len(Y_test)}")

Y_test and Y_pred matches in length? Answer: True


In [37]:
true_positives = get_true_positives(Y_test, Y_pred)
true_negatives = get_true_negatives(Y_test, Y_pred)
print(f"The number of true positives is: {true_positives}\nThe number of true negatives is: {true_negatives}")
accuracy = (true_positives + true_negatives)/len(Y_test)
print(f"Accuracy is: {accuracy:.4f}")

The number of true positives is: 249
The number of true negatives is: 723
Accuracy is: 0.8482


In [ ]:
print(f"The example email has: {len(treated_email)} words in the product.")

So the email you are investigating has $2657$ words! Let's compute the value $P(\text{word} \mid \text{ham})$ for the first 3 words in the email:

In [ ]:
for i in range(3):
    word = treated_email[i]
    p_word_given_ham = prob_word_given_class(word, cls = 'ham', word_frequency = word_frequency, class_frequency = class_frequency)
    print(f"Word: {word}. P({word} | ham) = {p_word_given_ham}")

Given that they are all probabilities, they are numbers between $0$ and $1$. So, the product being performed is a product of $2657$ numbers between $0$ and $1$. In the best-case scenario, where every word has a probability in the magnitude of $10^{-1}$ (similar to the first word in the example above), the resulting probability would be in the magnitude of $10^{-2657}$—a **very small number** that is challenging for any computer to handle with precision. Let's examine Python's limit on floating-point numbers (decimal numbers):

In [ ]:
import sys

print(sys.float_info)

As you can see, the minimum float value has a magnitude of $10^{-308}$, significantly larger than $10^{-2657}$. Consequently, Python interprets the result of the product as $0$ at some point, leading to the loss of all information. In other words, the way your algorithm is currently written, past a certain length, all emails are being classified as spam. Given the nature of this issue, rooted in the very large product required by Naive Bayes, it is crucial to address the problem.

#### 5.1.1 The Underflow Problem

The challenge you encounter is termed an **underflow problem**, indicating that you are dealing with exceedingly small numbers beyond the computer's precision. In this case, the root cause is the **very large product** involved in Naive Bayes calculations. Fortunately, there is a solution to this issue.

Recall that in Naive Bayes, the specific values of probabilities are not critical since the algorithm solely **compares values**. This is why the denominators in the following equations have been disregarded:

$$ P(\text{spam} \mid \text{email}) = \frac{P(\text{spam}) \cdot P(\text{email} \mid \text{spam})}{P(\text{email})} $$
$$ P(\text{ham} \mid \text{email}) = \frac{P(\text{ham}) \cdot P(\text{email} \mid \text{ham})}{P(\text{email}) } $$

Given that the goal is to identify the greater value between the two, and they share the same positive denominator, only the numerators matter. Specifically, the actual values of these two products:

$$P(\text{spam}) \cdot P(\text{email} \mid \text{spam})$$
$$P(\text{ham}) \cdot P(\text{email} \mid \text{ham})$$

are irrelevant, as long as you can tell which one is larger than the other.

If there exists a function that can be applied to these quantities and **preserves the ordering**, then comparing the outputs of these values in such a function will determine the class with the maximum value (although the actual numeric value may differ). 

Any **strictly increasing function** possesses this property: it preserves the maximum **point**. Therefore, the idea is to find a **increasing function** that aids in handling the large product faced by the Naive Bayes algorithm. Can you think of one? Well, there is one: the $\log$ function. As you may already know, $\log$ can transform **products** into **sums**! Since $\log$ is increasing, it preserves the maximum point. Therefore, you can compare the following quantities:

$$\log \left(P(\text{spam}) \cdot P(\text{email} \mid \text{spam}) \right)$$
$$\log \left(P (\text{ham}) \cdot P(\text{email} \mid \text{ham}) \right)$$

And choose the maximum value among these new quantities. Denoting the class as either spam or ham:

$$\log \left(P(\text{class}) \cdot P(\text{email} \mid \text{class}) \right) = \log \left(P(\text{class}) \right) + \log \left( P(\text{email} \mid \text{class}) \right)$$

And

$$\log \left( P(\text{email} \mid \text{class}) \right) = \log  \left(P(\text{word}_1 \mid \text{class}) \cdot P(\text{word}_2 \mid \text{class}) \cdots P(\text{word}_n \mid \text{class}) \right) = \log  \left(P(\text{word}_1 \mid \text{class}) \right) + \log \left(P(\text{word}_2 \mid \text{class})\right) + \cdots + \log \left( P(\text{word}_n \mid \text{class}) \right) $$

With this approach, you have transformed a large product into a large summation, a significantly more numerically stable operation. Now, you will improve our functions with this new technique! You need to adjust two functions:

- `prob_email_given_class` - replace the probability word product by the sum of the logs
- `naive_bayes` - replace the product $P(\text{class}) \cdot P(\text{email} \mid \text{class})$ by its respective sum of log.

The new functions will be called `log_prob_email_given_class` and `log_naive_bayes`.

In [ ]:
def log_prob_email_given_class(treated_email, cls, word_frequency, class_frequency):
    """
    Calculate the log probability of an email being of a certain class (e.g., spam or ham) based on treated email content.

    Parameters:
    - treated_email (list): A list of treated words in the email.
    - cls (str): The class label ('spam' or 'ham')
    

    Returns:
    - float: The log probability of the given email belonging to the specified class.
    """

    # prob starts at 0 because it will be updated by summing it with the current log(P(word | class)) in every iteration
    prob = 0

    for word in treated_email: 
        # Only perform the computation for words that exist in the word frequency dictionary
        if word in word_frequency.keys(): 
            # Update the prob by summing it with log(P(word | class))
            prob += np.log(prob_word_given_class(word, cls,word_frequency, class_frequency))

    return prob

In [ ]:
# Consider an email with only one word, so it reduces to compute the value P(word | class) or log(P(word | class)).
one_word_email = ['schedule']
word = one_word_email[0]
prob_spam = prob_email_given_class(one_word_email, cls = 'spam',word_frequency = word_frequency, class_frequency = class_frequency)
log_prob_spam = log_prob_email_given_class(one_word_email, cls = 'spam',word_frequency = word_frequency, class_frequency = class_frequency)
print(f"For word {word}:\n\tP({word} | spam) = {prob_spam}\n\tlog(P({word} | spam)) = {log_prob_spam}")

Note that the $\text{log}$ was capable of transforming a small number into a negative number with a good magnitude. Furthermore, now the algorithm is performing a sum instead of product.

The next code block implements the log_naive_bayes.

In [ ]:
def log_naive_bayes(treated_email, word_frequency, class_frequency, return_likelihood = False):    
    """
    Naive Bayes classifier for spam detection, comparing the log probabilities instead of the actual probabilities.

    This function calculates the log probability of an email being spam (1) or ham (0)
    based on the Naive Bayes algorithm. It uses the conditional probabilities of the
    treated_email given spam and ham, as well as the prior probabilities of spam and ham
    classes. The final decision is made by comparing the calculated probabilities.

    Parameters:
    - treated_email (list): A preprocessed representation of the input email.
    - return_likelihood (bool): If true, it returns the log_likelihood of both spam and ham.

    Returns:
    - int: 1 if the email is classified as spam, 0 if classified as ham.
    """
    
    # Compute P(email | spam) with the new log function
    log_prob_email_given_spam = log_prob_email_given_class(treated_email, cls = 'spam',word_frequency = word_frequency, class_frequency = class_frequency) 

    # Compute P(email | ham) with the function you defined just above
    log_prob_email_given_ham = log_prob_email_given_class(treated_email, cls = 'ham',word_frequency = word_frequency, class_frequency = class_frequency) 

    # Compute P(spam) using the class_frequency dictionary and using the formula #spam emails / #total emails
    p_spam = class_frequency['spam']/(class_frequency['ham'] + class_frequency['spam']) 

    # Compute P(ham) using the class_frequency dictionary and using the formula #ham emails / #total emails
    p_ham = class_frequency['ham']/(class_frequency['ham'] + class_frequency['spam']) 

    # Compute the quantity log(P(spam)) + log(P(email | spam)), let's call it log_spam_likelihood
    log_spam_likelihood = np.log(p_spam) + log_prob_email_given_spam 

    # Compute the quantity P(ham) * P(email | ham), let's call it ham_likelihood
    log_ham_likelihood = np.log(p_ham) + log_prob_email_given_ham 

    # In case of passing return_likelihood = True, then return the desired tuple
    if return_likelihood == True:
        return (log_spam_likelihood, log_ham_likelihood)
    
    # Compares both values and choose the class corresponding to the higher value. 
    # As the logarithm is an increasing function, the class with the higher value retains this property.
    if log_spam_likelihood >= log_ham_likelihood:
        return 1
    else:
        return 0

Revisiting the example from the beginning of the section, you will compute `log_spam_likelihood` and `log_ham_likelihood`

In [ ]:
log_spam_likelihood, log_ham_likelihood = log_naive_bayes(treated_email,word_frequency = word_frequency, class_frequency = class_frequency,return_likelihood = True)
print(f"log_spam_likelihood: {log_spam_likelihood}\nlog_ham_likelihood: {log_ham_likelihood}")

Great! Now there are two distinct non-zero numbers! Note the higher one is the `log_ham_likelihood`, therefore the `log_naive_bayes` function will correctly predict this email as ham:

In [ ]:
print(f"The example email is labeled as: {Y[example_index]}")
print(f"Log Naive bayes model classifies it as: {log_naive_bayes(treated_email,word_frequency = word_frequency, class_frequency = class_frequency)}")

With this enhanced algorithm, the new accuracy is:

In [ ]:
# Let's get the predictions for the test set:

# Create an empty list to store the predictions
Y_pred = []


# Iterate over every email in the test set
for email in X_test:
    # Perform prediction
    prediction = log_naive_bayes(email,word_frequency = word_frequency, class_frequency = class_frequency)
    # Add it to the list 
    Y_pred.append(prediction)

# Get the number of true positives:
true_positives = get_true_positives(Y_test, Y_pred)

# Get the number of true negatives:
true_negatives = get_true_negatives(Y_test, Y_pred)

print(f"The number of true positives is: {true_positives}\nThe number of true negatives is: {true_negatives}")

# Compute the accuracy by summing true negatives with true positives and dividing it by the total number of elements in the dataset. 
# Since both Y_pred and Y_test have the same length, it does not matter which one you use.
accuracy = (true_positives + true_negatives)/len(Y_test)

print(f"The accuracy is: {accuracy:.4f}")

This is a **huge** improvement! You've increased the model's accuracy from 84.82% to 99.21% in the test set! An increase of almost 17%. And you haven't touched the dataset, it was an improvement purely in the **math** behind it. Powerful, right?

<a name="5.2"></a>
### 5.2 Enhancing model performance: Practical implementation with Naive Bayes

#### 5.2.1 Introduction

In this section you will use both Naive Bayes models (with and without log) you've defined above to solve a problem:

You must develop a good spam detection model to run in a specific email software. The dataset you worked with in this assignment is the email base you have from this software. You must build a method to effectively protect users from receiving spam, **but you must avoid sending ham emails to the spam folder** since it might cause a user to lose important emails. On the other hand, it is not that concerning letting pass a couple of spam emails to the inbox folder. 

#### 5.2.2 Accuracy and its limitations

Right now, what is the actual performance of the model you've developed thus far? The accuracy metric you defined above has some limitations, specially in this spam detection problem. You have seen in the beginning of the notebook that the proportion of spam emails in the dataset is 23.88%. So, if you create a rule to send **every email directly to inbox folder** it would correctly classify 76.12% of every email! So this pointless rule has an accuracy of 76.12%. 

To try to properly answer this question, you can ask yourself two questions:

- How many spam emails the algorithm correctly classifies as spam? They are called **true positives**.
- How many **ham** emails the algorithm **mistakenly classifies** as spam? They are called **false positives**. **This is the important question you must look closer.**

The first question relates to a metric called [*recall*](https://en.wikipedia.org/wiki/Precision_and_recall). To answer the first question, you must count how many spam emails there exist in the dataset and count how many of them are correctly labeled as spam by the model (true positives). This is defined as the recall:

$$\text{recall} = \frac{\text{true positives (spam emails correctly labeled as spam)}}{\text{every spam email}}$$

Another way you may see this metric being defined is by considering that a spam email will be either correctly labeled as spam (true positive) or mistakenly labeled as ham (false negative), so 

$$\text{recall} =\frac{\text{true positives}}{\text{true positives} + \text{false negatives}}$$

You will now make the recall function.

In [ ]:
def get_recall(Y_true, Y_pred):
    """
    Calculate the recall for a binary classification task.

    Parameters:
    - Y_true (array-like): Ground truth labels.
    - Y_pred (array-like): Predicted labels.

    Returns:
    - recall (float): The recall score, which is the ratio of true positives to the total number of actual positives.
    """
    # Get the total number of spam emails. Since they are 1 in the data, it suffices summing all the values in the array Y.
    total_number_spams = Y_test.sum()
    # Get the true positives
    true_positives = get_true_positives(Y_true, Y_pred)
    
    # Compute the recall
    recall = true_positives/total_number_spams
    return recall

In [ ]:
# Use the Naive Bayes model (standard and log versions) to classify every email in the test dataset
Y_pred_naive_bayes = []
Y_pred_log_naive_bayes = []

for email in X_test:
 prediction = naive_bayes(email,word_frequency = word_frequency, class_frequency = class_frequency)
 log_prediction = log_naive_bayes(email,word_frequency = word_frequency, class_frequency = class_frequency)
 Y_pred_naive_bayes.append(prediction)
 Y_pred_log_naive_bayes.append(log_prediction)

# Compute the recall for both models
recall_naive_bayes = get_recall(Y_test, Y_pred_naive_bayes)
recall_log_naive_bayes = get_recall(Y_test, Y_pred_log_naive_bayes)

In [ ]:
print(f"The proportion of spam emails the standard Naive Bayes model can correctly classify as spam (recall) is: {recall_naive_bayes:.4f}")
print(f"The proportion of spam emails the log Naive Bayes model can correctly classify as spam (recall) is: {recall_log_naive_bayes:.4f}")

Ok, both models perform pretty well in **detecting spams**, being able to correctly identify 98% of them! This metric tells us about the model's **sensitivity**. In other words, this metric shows us how effective the model is in detecting a spam email.

Now you are left with the second question. It is related to another metric called *[precision](https://en.wikipedia.org/wiki/Precision_and_recall)*. 

To answer the question you must look at all emails the Naive Bayes models classify as spam and, in that pool, how many are **in fact** spam? This is an important metric to look, because a model that classifies any email as spam is a model that correctly classifies 100% of the spam emails, however it is pointless! Furthermore, you must avoid sending regular emails to the spam folder, otherwise the users may lose important emails.

This question is related to what it is called **false positives**. In other words, now you are looking at how many ham emails the algorithm sends to the spam folder. In the next code block, you will build a function to compute the false positives.

In [ ]:
def get_false_positives(Y_true, Y_pred):
    """
    Calculate the number of false positives instances in binary classification.

    Parameters:
    - Y_true (list): List of true labels (0 or 1) for each instance.
    - Y_pred (list): List of predicted labels (0 or 1) for each instance.

    Returns:
    - int: Number of false positives, where true label is 0 and predicted label is 1.
    """
    
    # Both Y_true and Y_pred must match in length.
    if len(Y_true) != len(Y_pred):
        return "Number of true labels and predict labels must match!"
    n = len(Y_true)

    false_positives = 0
    # Iterate over the number of elements in the list
    for i in range(n):
        # Get the true label for the considered email
        true_label_i = Y_true[i]
        # Get the predicted (model output) for the considered email
        predicted_label_i = Y_pred[i]
        # Increase the counter by 1 only if true_label_i = 0 and predicted_label_i = 0 (false positive)
        if true_label_i == 0 and predicted_label_i == 1:
            false_positives += 1
    return false_positives

In [ ]:
# Count the ham emails mistakenly labeled as spam (false positives). Let's use the function get_false_positives you've seen above
 
false_positives_naive_bayes = get_false_positives(Y_test, Y_pred_naive_bayes)
false_positives_log_naive_bayes = get_false_positives(Y_test, Y_pred_log_naive_bayes)

In [ ]:
print(f"Number of false positives in the standard Naive Bayes model: {false_positives_naive_bayes}")
print(f"Number of false positives in the log Naive Bayes model: {false_positives_log_naive_bayes}")

This is a huge improvement! You went from 169 ham emails being mistakenly labeled as spam to only 4! To get a more meaningful number, you can compute the following quantity: 

- The proportion of actual spam emails (true positives) that exists in the pool of predicted spam emails. Note that the pool of predicted emails consist of **every spam email correctly labeled as spam** (true positives) and **every ham email mistakenly labeled as spam** (false positives).

This quantity is called **precision** and it is defined as:

$$\text{precision} = \frac{\text{true positives}}{\text{true positives} + \text{false positives}}$$

This metric tells you how **relevant** the output of your model is. As already discussed, a model that predicts every email as spam can correctly identify every spam email, however its output is irrelevant since it sends every ham email to the spam folder. You will now implement it.

In [ ]:
def get_precision(Y_true, Y_pred):
    """
    Calculate precision, a metric for the performance of a classification model,
    by computing the ratio of true positives to the sum of true positives and false positives.

    Parameters:
    - Y_true (list): True labels.
    - Y_pred (list): Predicted labels.

    Returns:
    - precision (float): Precision score.
    """
    # Get the true positives
    true_positives = get_true_positives(Y_true, Y_pred)
    false_positives = get_false_positives(Y_true, Y_pred)
    precision = true_positives/(true_positives + false_positives)
    return precision

In [ ]:
print(f"Precision of the standard Naive Bayes model: {get_precision(Y_test, Y_pred_naive_bayes):.4f}")
print(f"Precision of the log Naive Bayes model: {get_precision(Y_test, Y_pred_log_naive_bayes):.4f}")

The first version of the model has a precision of 59.57%. In other words, from 100 emails the model classifies as spam, only around 60 of them are in fact spam. This means that this model would send 40 ham emails to the spam folder, indicating that, even though very sensitive, it is not very reliable. 

On the other hand, the improved model has a precision of 98.42%! So from 100 emails classified as spam, only around 2 will be actually ham emails. A much more reliable output. 

Congratulations! You have completed the entire assignment and the appendix section! 